# Exercise 1: Graph Making

Write a function called make_simple_graph that returns the graph object that represents the following diagram:

![](exercise2.png)

In [3]:
class Make_Simple_graph:
    def __init__(self):
        self.adjency_list = {}

    def add_vertex(self, vertex):
        if(vertex in self.adjency_list):
            raise Exception('Vertex already exists')
        self.adjency_list[vertex] = []

    def add_edge(self, v1, v2):
        self.adjency_list[v1] = [*self.adjency_list[v1], v2]
        self.adjency_list[v2] = [*self.adjency_list[v2], v1]

    def get_adjency_list(self):
        return (self.adjency_list)

    def depth_first_recursive(self, start):
        result = []
        visited = {}
        def dfs(vertex):
            if (vertex is None):
                return None
            
            visited[vertex] = True
            result.append(vertex)
            for neighbor in self.adjency_list[vertex]:
                if(visited.get(neighbor) is None):
                    return dfs(neighbor)
        dfs(start)
        return result

    def breath_first_search(start):
        queue = [start]
        result = []
        visited = {}

        while(len(queue) is not 0):
            current_vertex = q

# instantiate graph
graph = Make_Simple_graph()
# create nodes
for vertex in ['A', 'B', 'C', 'D', 'E']:
    graph.add_vertex(vertex)

for edge in [('B', 'D'), ('D', 'E'), ('E', 'C'), ('C', 'A'), ('A', 'E'), ('A' 'D')]:
    graph.add_edge(edge[0], edge[1])

graph.get_adjency_list()



{'A': ['C', 'E', 'D'],
 'B': ['D'],
 'C': ['E', 'A'],
 'D': ['B', 'E', 'A'],
 'E': ['D', 'C', 'A']}

## Exercise 1.2: computing diameter and node degrees of graphs

For the graph made by your function in the previous exercise, calculate the diameter and the degrees of each node, and visually confirm those values by inspecting the above graph. Write a function `compute_diameter_and_degrees` which takes a networkx graph object as input, and returns a `dict` with the diameter and the degrees of all the nodes in the graph.

**NOTE:** You cannot use a networkx function or method to compute the diameter or degrees. They have to be calculated yourself

```

compute_diameter_and_degrees(G) -> {
    'diameter': 3,
    'degree_A': 3,
    'degree_B': 1,
    'degree_C': 2,
    'degree_D': 3,
    'degree_E': 3,
}

In [15]:
import networkx as nx
from itertools import combinations

def compute_diameter_and_degrees(g):
    G = nx.Graph(g.get_adjency_list())
    # generate all combination of nodes. assuming connected graph
    all_pairs = list(combinations(g.get_adjency_list().keys(), 2))
    # diameter is longest shortest path
    result = {
        'diameter': max([nx.shortest_path_length(G, pair[0], pair[1]) for pair in all_pairs])
    }
    # degree is all verteces
    for vertex in g.get_adjency_list().keys():
        result[f'degree_{vertex}'] = len(g.get_adjency_list()[vertex])
    return result

compute_diameter_and_degrees(graph)

{'diameter': 3,
 'degree_A': 3,
 'degree_B': 1,
 'degree_C': 2,
 'degree_D': 3,
 'degree_E': 3}

## Exercise 1.3: An airline route map graph

CBDS Airlines has the following route map for the cities that it services, where every pair of cities has service in both directions between the cities:

```
routemap =  [('St. Louis', 'Miami'), ('St. Louis', 'San Diego'), ('St. Louis', 'Chicago'), ('San Diego', 'Chicago'), ('San Diego', 'San Francisco'), ('San Diego', 'Minneapolis'), ('San Diego', 'Boston'), ('San Diego', 'Portland'), ('San Diego', 'Seattle'), ('Tulsa', 'New York'), ('Tulsa', 'Dallas'), ('Phoenix', 'Cleveland'), ('Phoenix', 'Denver'), ('Phoenix', 'Dallas'), ('Chicago', 'New York'), ('Chicago', 'Los Angeles'), ('Miami', 'New York'), ('Miami', 'Philadelphia'), ('Miami', 'Denver'), ('Boston', 'Atlanta'), ('Dallas', 'Cleveland'), ('Dallas', 'Albuquerque'), ('Philadelphia', 'Atlanta'), ('Denver', 'Minneapolis'), ('Denver', 'Cleveland'), ('Albuquerque', 'Atlanta'), ('Minneapolis', 'Portland'), ('Los Angeles', 'Seattle'), ('San Francisco', 'Portland'), ('San Francisco', 'Seattle'), ('San Francisco', 'Cleveland'), ('Seattle', 'Portland')]

```

Copy this data into a networkX graph to answer the question.

1. What is the maximum number of hops that would ever be taken by a passenger on a single trip between any two serviced cities?

2. If you were a rich jet-setter traveling everywhere in across the country and were constrained to fly CBDS, which city would be the most optimal place for you to live, to minimize the number of hops you would have to make on average as you jet from home to your latest vacation spot?


In [32]:
import networkx as nx
import operator

routemap = [('St. Louis', 'Miami'), ('St. Louis', 'San Diego'), ('St. Louis', 'Chicago'), ('San Diego', 'Chicago'), ('San Diego', 'San Francisco'), ('San Diego', 'Minneapolis'), ('San Diego', 'Boston'), ('San Diego', 'Portland'), ('San Diego', 'Seattle'), ('Tulsa', 'New York'), ('Tulsa', 'Dallas'), ('Phoenix', 'Cleveland'), ('Phoenix', 'Denver'), ('Phoenix', 'Dallas'), ('Chicago', 'New York'), ('Chicago', 'Los Angeles'), ('Miami', 'New York'), ('Miami', 'Philadelphia'), ('Miami', 'Denver'), ('Boston', 'Atlanta'), ('Dallas', 'Cleveland'), ('Dallas', 'Albuquerque'), ('Philadelphia', 'Atlanta'), ('Denver', 'Minneapolis'), ('Denver', 'Cleveland'), ('Albuquerque', 'Atlanta'), ('Minneapolis', 'Portland'), ('Los Angeles', 'Seattle'), ('San Francisco', 'Portland'), ('San Francisco', 'Seattle'), ('San Francisco', 'Cleveland'), ('Seattle', 'Portland')]

G = nx.DiGraph(routemap)

longest_path = nx.algorithms.dag.dag_longest_path_length(G)
centrality = nx.degree_centrality(G)

print(f'The highest number of hops is {longest_path}')
print(f'the most central location is {max(centrality.items(), key=operator.itemgetter(1))[0]}')

The highest number of hops is 5
the most central location is San Diego


# Exercise 2

Make an algorithm to generate [random walks](https://en.wikipedia.org/wiki/Random_walk) on a NetworkX graphs.

The form is `random_walk(G, nodeid, length)` so the input should be a graph, node ID and the number of steps to take in the random walk. The output is a list of node IDs in the walks

```

G = nx.binomial_graph(25, 0.3, directed=True)

# this output is random
# don't try to reproduce it exactly
random_walk(G, 3, 5) -> [3, 1, 4, 2, 3]
```


In [50]:
import networkx as nx
import random

def random_walk(g, nodeid, length):
    G = nx.Graph(g.get_adjency_list())

    current_node = nodeid
    walk = [current_node]

    for i in range(length - 1):
        step = random.choice(list(G.edges(current_node)))[1]
        walk = [*walk, step]
        current_node = step
    return walk

random_walk(graph, 'B', 5)

['B', 'D', 'B', 'D', 'A']

## Exercise 2.2

Modify your random walk algorithm so that:

- It takes in on a weighed graph's adjacency matrix (a numpy matrix). 

- The probability to go from node A to node B should be proportional to the weight on their edge (relative to the other edges starting at node A).

Example:
```
G = np.array([
    [0, 0.5, 0, 0.5],
    [0.5, 0, 0, 0.5],
    [0.25, 0.25, 0.25, 0.25],
    [0.5, 0.5, 0, 0],
])

random_walk(G, 0, 4) -> [0, 1, 0, 3]
```

Here, `node 0` would give us 50% chance to transition to `node 1` and 50% chance to transition to `node 3`.

Then `node 1` would give us 50% chance to transition to `node 0` and 50% to `node 3`, etc.

In [67]:
import numpy as np

def random_walk(g, start, length):
    current_node = start
    walk = [current_node]
    for i in range(length - 1):
        step = np.random.choice([i for i in range(length)], 1,
              p=g[current_node])[0]
        walk = [*walk, step]
        current_node = step
            
    return walk

G = np.array([
    [0, 0.5, 0, 0.5],
    [0.5, 0, 0, 0.5],
    [0.25, 0.25, 0.25, 0.25],
    [0.5, 0.5, 0, 0],
])

random_walk(G, 0, 4) 


[0, 1, 0, 3]


# (stretch) Exercise 3: Dijkstra's Shortest Path Algorithm

Dijkstra's algorithm uses breadth first search to find the **shortest path** between two nodes in a graph.

It goes like this:

----

Let the node at which we are starting be called the initial node. Let the distance of node Y be the distance from the initial node to Y. Dijkstra's algorithm will assign some initial distance values and will try to improve them step by step.

1. Mark all nodes unvisited. Create a set of all the unvisited nodes called the unvisited set.

2. Assign to every node a tentative distance value: set it to zero for our initial node and to infinity for all other nodes. Set the initial node as current.

3. For the current node, consider all of its unvisited neighbours and calculate their tentative distances through the current node. Compare the newly calculated tentative distance to the current assigned value and assign the smaller one. For example, if the current node A is marked with a distance of 6, and the edge connecting it with a neighbour B has length 2, then the distance to B through A will be 6 + 2 = 8. If B was previously marked with a distance greater than 8 then change it to 8. Otherwise, the current value will be kept.

4. When we are done considering all of the unvisited neighbours of the current node, mark the current node as visited and remove it from the unvisited set. A visited node will never be checked again.

5. If the destination node has been marked visited (when planning a route between two specific nodes) or if the smallest tentative distance among the nodes in the unvisited set is infinity (when planning a complete traversal; occurs when there is no connection between the initial node and remaining unvisited nodes), then stop. The algorithm has finished.

6. Otherwise, select the unvisited node that is marked with the smallest tentative distance, set it as the new "current node", and go back to step 3.

----

Here is a GIF of it in action:

![](Dijkstras_progress_animation.gif)

Implement `shortest_path(G, start_node, end_node)` which takes in a networkX graph, a starting node and an end node and outputs a list of nodes which is the shortest path between the start and end.

You can use networkX's shortest path function to test your result, but not to implement it.